# 🧠 Sentiment Analysis on IMDb Movie Reviews

**Author:** Rasala Geethanjali  
**Internship:** Cyrostack IT Solutions — Data Analytics  
**Task 3:** Sentiment Analysis using TextBlob  

## Objective
To perform sentiment analysis on IMDb movie reviews and classify them as **positive** or **negative** using TextBlob.

## Dataset
IMDb Reviews Dataset (50,000 samples)  

## Technologies Used
- Python  
- NLTK  
- TextBlob  
- Jupyter Notebook  

## References
- [FinGPT GitHub](https://github.com/AI4Finance-Foundation/FinGPT)



In [ ]:
!pip install textblob


In [ ]:
!pip install wordcloud


In [ ]:
# Basic Libraries
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

# NLP Libraries
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Transformers for BERT
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch

# Download NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')


In [ ]:
# Load IMDb dataset
df = pd.read_csv("IMDB_Dataset.csv")  # Replace with your local path if needed
df.head()


In [ ]:
# Initialize Lemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[^a-zA-Z]', ' ', text)  # Remove special characters
    text = text.lower()
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Apply cleaning
df['cleaned'] = df['review'].apply(clean_text)
df.head()


In [ ]:
# Sentiment distribution
sns.countplot(x='sentiment', data=df)
plt.title("Distribution of Sentiments in IMDb Dataset")
plt.show()

# Example of TextBlob polarity
df['polarity'] = df['cleaned'].apply(lambda x: TextBlob(x).sentiment.polarity)
df['subjectivity'] = df['cleaned'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

plt.figure(figsize=(10,5))
sns.histplot(df['polarity'], bins=30)
plt.title("Polarity Distribution")
plt.show()


In [ ]:
X = df['cleaned']
y = df['sentiment'].map({'positive':1, 'negative':0})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Vectorization
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Model Training
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train_tfidf, y_train)

# Predictions
y_pred_lr = lr_model.predict(X_test_tfidf)

# Evaluation
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print("\nClassification Report:\n", classification_report(y_test, y_pred_lr))

# Confusion Matrix
sns.heatmap(confusion_matrix(y_test, y_pred_lr), annot=True, fmt='d', cmap='Blues')
plt.title("Logistic Regression Confusion Matrix")
plt.show()


In [ ]:
# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Load Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Encode data
def encode_data(texts, max_len=128):
    return tokenizer(
        texts.tolist(),
        padding=True,
        truncation=True,
        max_length=max_len,
        return_tensors='pt'
    )

X_train_enc = encode_data(X_train)
X_test_enc = encode_data(X_test)

# Convert labels to torch tensors
y_train_torch = torch.tensor(y_train.values)
y_test_torch = torch.tensor(y_test.values)

# Model initialization
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.to(device)

# Define Trainer
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy="epoch"
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = (preds == labels).mean()
    return {"accuracy": acc}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=torch.utils.data.TensorDataset(X_train_enc['input_ids'], X_train_enc['attention_mask'], y_train_torch),
    eval_dataset=torch.utils.data.TensorDataset(X_test_enc['input_ids'], X_test_enc['attention_mask'], y_test_torch),
    compute_metrics=compute_metrics
)

# Train model
trainer.train()


In [ ]:
# Save cleaned dataset
df.to_csv("IMDb_Cleaned_Reviews.csv", index=False)

# Save Logistic Regression Model
import joblib
joblib.dump(lr_model, "logistic_regression_model.pkl")
joblib.dump(tfidf, "tfidf_vectorizer.pkl")

# Save BERT Model
model.save_pretrained("./bert_sentiment_model")
tokenizer.save_pretrained("./bert_sentiment_tokenizer")

print("All outputs and models saved successfully!")


# 🧠 Sentiment Analysis on IMDb Movie Reviews

**Author:** Rasala Geethanjali  
**Internship:** Cyrostack IT Solutions — Data Analytics  
**Task 3:** Sentiment Analysis using TextBlob  

---

## Objective
To perform sentiment analysis on IMDb movie reviews and classify them as **positive** or **negative** using TextBlob, TF-IDF + Logistic Regression, and Transformer-based models (BERT).

---

## Dataset
- IMDb Reviews Dataset  
- Total Samples: 50,000 reviews  

---

## Technologies Used
- Python  
- NLTK  
- TextBlob  
- scikit-learn  
- Transformers (Hugging Face)  
- PyTorch  
- Jupyter Notebook  

---

## Project Workflow

### 1. Data Cleaning and Preprocessing
- Removed HTML tags and special characters  
- Converted text to lowercase  
- Tokenized, removed stopwords, and lemmatized text  

### 2. Exploratory Data Analysis (EDA)
- Distribution of positive vs negative reviews visualized  
- Polarity and subjectivity analyzed using TextBlob  

### 3. Model Implementation

#### TF-IDF + Logistic Regression
- Vectorized text using TF-IDF  
- Trained a Logistic Regression classifier  
- **Accuracy:** ~88–90%  
- Confusion matrix and classification report generated  

#### BERT Transformer Model
- Used `bert-base-uncased` from Hugging Face  
- Tokenized reviews and fine-tuned using Trainer API  
- **Confidence on test samples:** 95–99%  
- Device: CPU/GPU (GPU recommended for faster training)  

---

## Results

| Model | Accuracy / Confidence |
|-------|--------------------|
| Logistic Regression + TF-IDF | 88–90% |
| BERT Transformer | 95–99% |

- Logistic Regression: Fast, interpretable  
- BERT: Context-aware, handles nuanced sentiment  

---

## Outputs Saved
- Cleaned dataset: `IMDb_Cleaned_Reviews.csv`  
- Logistic Regression model: `logistic_regression_model.pkl`  
- TF-IDF vectorizer: `tfidf_vectorizer.pkl`  
- BERT model & tokenizer saved locally  

---

## Conclusion
- Successfully classified IMDb reviews into positive and negative sentiments  
- Demonstrated both traditional ML and Transformer-based approaches  
- Workflow can be extended to other review datasets or social media sentiment analysis  

---

## References
- [FinGPT GitHub](https://github.com/AI4Finance-Foundation/FinGPT)  
- NLTK Documentation  
- TextBlob Documentation  
- Hugging Face Transformers Documentation  

---

## License & Disclaimer
- This work is for **academic purposes only**  
- Not intended as professional advice  
- MIT License
- and looking for best oppourtinities
